In [1]:
# Install the necessary libraries
!pip install gdown

# Download the dataset zip file from Google Drive
!gdown --id 1Muvn7s13q0--kv2bIpPDh9M18QyY_OOP -O dataset.zip

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Muvn7s13q0--kv2bIpPDh9M18QyY_OOP
To: /kaggle/working/dataset.zip
100%|███████████████████████████████████████| 2.91G/2.91G [00:21<00:00, 137MB/s]


In [2]:
!unzip "/kaggle/working/dataset.zip"

Archive:  /kaggle/working/dataset.zip
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000001_000.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000001_001.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000001_002.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000002_000.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_000.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_001.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_002.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_003.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_004.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_005.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_006.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000003_007.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/00000004_000.png  
 extracting: Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/0

In [3]:
import os
import numpy as np
import random
import cv2
from sklearn.model_selection import train_test_split

# Define the directory paths for the positive and negative pneumonia images
pneumonia_positive_dir = '/kaggle/working/Data_Pneumonia/PNEUMONIA_POSITIVE'
pneumonia_negative_dir = '/kaggle/working/Data_Pneumonia/PNEUMONIA_NEGATIVE (1)'

# Define the image size
img_size = 224

# Create empty arrays to store the images and labels
images = []
labels = []

# Load the positive pneumonia images
for filename in os.listdir(pneumonia_positive_dir):
    img = cv2.imread(os.path.join(pneumonia_positive_dir, filename))
    img = cv2.resize(img, (img_size, img_size))
    images.append(img)
    labels.append(1)  # Positive label

# Load the negative pneumonia images
for filename in os.listdir(pneumonia_negative_dir):
    img = cv2.imread(os.path.join(pneumonia_negative_dir, filename))
    img = cv2.resize(img, (img_size, img_size))
    images.append(img)
    labels.append(0)  # Negative label

# Convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the dataset
random.seed(42)
idx = np.arange(len(images))
random.shuffle(idx)
images = images[idx]
labels = labels[idx]

# Divide the dataset into training, testing, and validation sets using an 80/10/10 split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1111, random_state=42, stratify=y_train)

# Print the size of each set
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Testing set size:", X_test.shape[0])


Training set size: 8000
Validation set size: 1000
Testing set size: 1000


In [4]:

from keras.preprocessing.image import ImageDataGenerator

# Define the augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Create the training generator
train_generator = datagen.flow(X_train, y_train, batch_size=32)


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

!pip install keras-tuner

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
import kerastuner as kt
from kerastuner import HyperParameters
from tensorflow.keras.applications import InceptionV3


# Define the build function
def build_model(hp):
    img_size = 224
    inc_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

    # Freeze the layers of the pre-trained models
    for layer in inc_model.layers:
        layer.trainable = False

    # Define the top layers of the models with hyperparameters
    inc_top_model = layers.Flatten()(inc_model.output)
    for i in range(hp.Int('dense_layers', 1, 3)):
        inc_top_model = layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu')(inc_top_model)
        inc_top_model = layers.Dropout(rate=hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.1))(inc_top_model)

    inc_top_model = layers.Dense(1, activation='sigmoid')(inc_top_model)

    # Combine the models
    inc_model_combined = tf.keras.Model(inputs=inc_model.input, outputs=inc_top_model)

    # Compile the models
    inc_model_combined.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return inc_model_combined

# Define the hyperparameters to tune
tuner_hp = HyperParameters()
tuner_hp.Int('dense_layers', 1, 3)
for i in range(3):
    tuner_hp.Int('units_' + str(i), min_value=32, max_value=512, step=32)
    tuner_hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.1)

# Instantiate the tuner object
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     hyperparameters=tuner_hp,
                     directory='tuner_dir',
                     project_name='inc_tuner')

# Search for the best hyperparameters
tuner.search(train_generator, epochs=10, validation_data=(X_val, y_val))

# Retrieve the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]

# Use the best hyperparameters to build and train the model
inc_model_tuned = build_model(best_hp)


Trial 30 Complete [00h 16m 15s]
val_accuracy: 0.5730000138282776

Best val_accuracy So Far: 0.609000027179718
Total elapsed time: 03h 29m 41s


In [8]:
inc_history_tuned = inc_model_tuned.fit(train_generator, epochs=10, validation_data=(X_val, y_val))
inc_score_tuned = inc_model_tuned.evaluate(X_val, y_val, verbose=0)

Epoch 1/10
250/250 [==============================] - 101s 382ms/step - loss: 2.2093 - accuracy: 0.8155 - val_loss: 8.6434 - val_accuracy: 0.5730
Epoch 2/10
250/250 [==============================] - 93s 372ms/step - loss: 0.3264 - accuracy: 0.8591 - val_loss: 5.8306 - val_accuracy: 0.4460
Epoch 3/10
250/250 [==============================] - 93s 372ms/step - loss: 0.3276 - accuracy: 0.8580 - val_loss: 1.4672 - val_accuracy: 0.6130
Epoch 4/10
250/250 [==============================] - 94s 376ms/step - loss: 0.3446 - accuracy: 0.8434 - val_loss: 7.1466 - val_accuracy: 0.5750
Epoch 5/10
250/250 [==============================] - 93s 374ms/step - loss: 0.3096 - accuracy: 0.8583 - val_loss: 27.8001 - val_accuracy: 0.5730
Epoch 6/10
250/250 [==============================] - 93s 374ms/step - loss: 0.3402 - accuracy: 0.8543 - val_loss: 23.7577 - val_accuracy: 0.5740
Epoch 7/10
250/250 [==============================] - 93s 373ms/step - loss: 0.3083 - accuracy: 0.8661 - val_loss: 26.5231 - va

In [9]:
#On validation set

print("InceptionV3 model results:")
print("Loss: ", inc_score_tuned[0])
print("Accuracy: ", inc_score_tuned[1])

InceptionV3 model results:
Loss:  23.270933151245117
Accuracy:  0.5730000138282776


In [10]:
inc_model_tuned.save('inc_model_tuned_10epochs.h5')

In [11]:
#On separate test set

inc_score_tuned_test = inc_model_tuned.evaluate(X_test, y_test, verbose=0)
print("InceptionV3 model results:")
print("Loss: ", inc_score_tuned_test[0])
print("Accuracy: ", inc_score_tuned_test[1])

InceptionV3 model results:
Loss:  23.183610916137695
Accuracy:  0.5730000138282776


In [12]:
import random
import matplotlib.pyplot as plt
import numpy as np

# Load the trained model
from keras.models import load_model
model = load_model('inc_model_tuned_10epochs.h5')

def random_image_prediction():
  # Load the image
  import os
  image_names = os.listdir('/kaggle/working/Data_Pneumonia/PNEUMONIA_POSITIVE')

  # Select a random image file
  random_file = random.choice(image_names)

  # Load the image using the `cv2.imread()` function
  image = cv2.imread("/kaggle/working/Data_Pneumonia/PNEUMONIA_POSITIVE/" + random_file)

  # Resize the image
  image = cv2.resize(image, (224, 224))

  # Preprocess the image
  image = image / 255.0

  # Reshape the image to have a batch dimension
  image = np.expand_dims(image, axis=0)

  # Make predictions on the image
  pred = model.predict(image)

  # Print the predicted probability of the positive class
  print(pred[0][0])

  if (pred[0][0] > 0.5):
    print("Positive")
  else:
    print("Negative")


print('1.')
random_image_prediction()
print('')
print('2.')
random_image_prediction()
print('') 
print('3.')
random_image_prediction()


1.
1/1 [==============================] - 2s 2s/step
0.6784098
Positive

2.
1/1 [==============================] - 0s 33ms/step
0.90919393
Positive

3.
1/1 [==============================] - 0s 31ms/step
0.6784098
Positive


In [15]:
model = load_model('inc_model_tuned_10epochs.h5')

def random_image_prediction():
  # Load the image
  import os
  image_names = os.listdir('/kaggle/working/Data_Pneumonia/PNEUMONIA_NEGATIVE (1)')

  # Select a random image file
  random_file = random.choice(image_names)

  # Load the image using the `cv2.imread()` function
  image = cv2.imread("/kaggle/working/Data_Pneumonia/PNEUMONIA_NEGATIVE (1)/" + random_file)

  # Resize the image
  image = cv2.resize(image, (224, 224))

  # Preprocess the image
  image = image / 255.0

  # Reshape the image to have a batch dimension
  image = np.expand_dims(image, axis=0)

  # Make predictions on the image
  pred = model.predict(image)

  # Print the predicted probability of the positive class
  print(pred[0][0])

  if (pred[0][0] > 0.5):
    print("Positive")
  else:
    print("Negative")


print('1.')
random_image_prediction()
print('')
print('2.')
random_image_prediction()
print('') 
print('3.')
random_image_prediction()


1.
1/1 [==============================] - 1s 1s/step
0.0017048734
Negative

2.
1/1 [==============================] - 0s 35ms/step
0.0030059074
Negative

3.
1/1 [==============================] - 0s 31ms/step
0.013058686
Negative


### 